In [ ]:
example = """
Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0
""".strip().splitlines()

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Registers:
    A: int
    B: int
    C: int
    pointer: int = 0
    output: list[int] = field(default_factory=lambda: [])

In [ ]:
def read(input):
    registers = {}
    for line in input[:3]:
        register, val = line[9:].split(":")
        registers[register] = int(val)
    registers = Registers(**registers)
    program = list(map(int, input[-1][9:].split(",")))
    return program, registers

In [ ]:
from functools import wraps

def inject_op(func):
    @wraps(func)
    def wrapped(op, registers):
        match op:
            case 4:
                op = registers.A
            case 5:
                op = registers.B
            case 6:
                op = registers.C
            case 7:
                raise ValueError("Operand 7 should not exist")
        return func(op, registers)
    return wrapped

In [ ]:
class Ops:
    def __init__(self):
        self.ops = []
    
    def __getitem__(self, pos):
        return self.ops[pos]
    
    def __call__(self, func):
        self.ops.append(func)
        return func

ops = Ops()

In [ ]:
@ops
@inject_op
def adv(op, registers):
    registers.A = registers.A // (2 ** op)

@ops
def bxl(op, registers):
    registers.B = registers.B ^ op

@ops
@inject_op
def bst(op, registers):
    registers.B = op % 8

@ops
def jnz(op, registers):
    if registers.A:
        registers.pointer = op - 2

@ops
def bxc(op, registers):
    registers.B = registers.B ^ registers.C

@ops
@inject_op
def out(op, registers):
    registers.output.append(op % 8)

@ops
@inject_op
def bdv(op, registers):
    registers.B = registers.A // (2 ** op)

@ops
@inject_op
def cdv(op, registers):
    registers.C = registers.A // (2 ** op)

In [ ]:
def execute(program, registers):
    while registers.pointer < len(program) and registers.pointer % 2 == 0:
        ops[program[registers.pointer]](program[registers.pointer+1], registers)
        registers.pointer += 2
    print(",".join(map(str, registers.output)))

In [ ]:
execute(*read(example))

In [ ]:
input = """
Register A: 33024962
Register B: 0
Register C: 0

Program: 2,4,1,3,7,5,1,5,0,3,4,2,5,5,3,0
""".strip().splitlines()

In [ ]:
execute(*read(input))

In [ ]:
def execute2(program, registers, A=None):
    if A is not None:
        registers.A = A
    while registers.pointer < len(program) and registers.pointer % 2 == 0:
        ops[program[registers.pointer]](program[registers.pointer+1], registers)
        registers.pointer += 2

In [ ]:
example2 = """
Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0
""".strip().splitlines()

In [ ]:
import copy


oprogram, oregisters = read(input)
A = 1_000_001
while True:
    program = copy.deepcopy(oprogram)
    registers = copy.deepcopy(oregisters)
    execute2(program, registers, A)
    if program == registers.output:
        print(A)
        break
    A += 1